In [0]:
import json
import os
def remove_file_path(data, keys_names):
    if isinstance(data, dict):
        for key,value in data.items():
            if key in keys_names and isinstance(value,str):
                data[key] = os.path.basename(value)
            elif isinstance(value,dict):
                remove_file_path(value, keys_names)
            elif isinstance(value,list):
                for item in value:
                    if isinstance(item,dict):
                        remove_file_path(item, keys_names)
    return data

def prefix_folder_path(data,folder_path):
    if isinstance(data,dict):
        for key,value in data.items():
            if key in folder_path and isinstance(value,str):
                data[key] = os.path.join(folder_path[key],value)
            elif isinstance(value,dict):
                prefix_folder_path(value,folder_path)
            elif isinstance(value,list):
                for item in value:
                    if isinstance(item,dict):
                        prefix_folder_path(item,folder_path)
    return data

def correct_onboard_json(volume_path):
    combined_json = []
    keys_to_modify = ["source_path_dev", "source_schema_path", "bronze_table_path_dev", "bronze_data_quality_expectations_json_dev", "bronze_data_quality_expectations_json_prod", "bronze_table_path_prod", "bronze_table_path_dev", "bronze_quarantine_table_path_prod", "bronze_quarantine_table_path_dev","silver_table_path_dev", "silver_transformation_json_dev"]

    folder_path = {"source_path_dev": dbutils.widgets.get("source_path_dev"),
               "bronze_table_path_dev":dbutils.widgets.get("bronze_table_path_dev"), 
               "source_schema_path":dbutils.widgets.get("source_schema_path"), 
               "silver_table_path_dev":dbutils.widgets.get("silver_table_path_dev"),
               "filestore":dbutils.widgets.get("filestore"),
               "DDL_file_path":dbutils.widgets.get("DDL_file_path"),
               "bronze_database":dbutils.widgets.get("bronze_database"),
               "bronze_data_quality_expectations_json_dev":dbutils.widgets.get("bronze_data_quality_expectations_json_dev"),
               "bronze_data_quality_expectations_json_prod": dbutils.widgets.get("bronze_data_quality_expectations_json_prod"),
               "bronze_table_path_prod":dbutils.widgets.get("bronze_table_path_prod"),
               "bronze_quarantine_table_path_prod": dbutils.widgets.get("bronze_quarantine_table_path_prod"),
               "bronze_quarantine_table_path_dev": dbutils.widgets.get("bronze_quarantine_table_path_dev"),
               "silver_transformation_json_dev" : dbutils.widgets.get("silver_transformation_json_dev")               
               }


    for file_name in os.listdir(volume_path):
        if file_name.endswith('.json'):
            with open(os.path.join(volume_path,file_name)) as json_file:
                data = json.load(json_file)
                corrected_data = remove_file_path(data[0],keys_to_modify)
                prefix_path = prefix_folder_path(corrected_data,folder_path)
                combined_json.append(prefix_path)
    
    return combined_json


In [0]:
combined_json = correct_onboard_json(os.path.join(dbutils.widgets.get("volume_path"), dbutils.widgets.get("bronze_database")))

with open(os.path.join(dbutils.widgets.get("volume_path"),dbutils.widgets.get("combined_json_file_name")), 'w') as outfile:
    json.dump(combined_json, outfile)

    